In [79]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [80]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [81]:
# Open Sheet
sheet = client.open_by_key("1n5E04Co2_zNYpX4lFI7UTER0tvlSl-AKHzd7YwhD8ok") # Master Sheets ( Dimensions Sheet)

In [82]:
StoreMaster = sheet.worksheet("Store Master")
StoreMaster = get_as_dataframe(StoreMaster)

In [83]:
# StoreMaster.head()

In [84]:
# list(CTReportExportAI.columns)

In [85]:
CTReportExportAI = pd.read_excel(r"C:\Users\Administrator\LIS\GCPL SAMT AI\CTReportExportAIV3_06-10-2025.xlsx")

In [86]:
CTReportExportAI = CTReportExportAI[['Visit ID','Submitted Date&Time','Sync Date&Time','Month','Year',
                                     'SupervisorCode','UserName','FullName',
                                     'Client Store Code','LIS Store Code','Store Name',
                                     'Store Address','Chain Code','Store Type','PJP Status',
                                     'Category Tracking Code','Category Tracking Name','Category Product Code',
                                     'Category Product Name','Is category Available','Reason for Non availability',
                                     'Image capturing enabled','AI Detection Status','AI detection reason',
                                     'Facings','Total Facings','Self Product Facings Detected',
                                     'Competition Products Detected','Total Facings Detected','Audit Status']]

In [87]:
# CSV Read Module
# CTReportExportAI = pd.read_excel(r"C:\Users\Administrator\LIS\GCPL SAMT AI\CTAIReportExport_22-06-2025.xlsx")
# CTReportExportAI= pd.read_excel(r'C:\Users\Administrator\LIS\GCPL SAMT AI\CTReportExportAI_21-05-2025.xls')
CTReportExportAI.rename(columns= {'Submitted Date&Time':'Date','Sync Date&Time':'SyncDate&Time','LISStoreCode ':'LISStoreCode','Category Tracking Code':'CategoryTrackingCode','Chain Name':'ChainName',
                                  'Reason for Non availability' :'Non Availability Reason','AI detection reason' : 'AIdetectionreason',
                                 'AI Detection Status': 'AIDetectionStatus'}, inplace=True)

In [88]:
CTReportExportAI_1 = CTReportExportAI[
    (
        CTReportExportAI['AIDetectionStatus'].isin(['Yes', 'No']) | 
        CTReportExportAI['AIDetectionStatus'].isna()
    ) &
    CTReportExportAI['CategoryTrackingCode'].isin([
        'Premium-01', 'SuperPremium-01'
    ])
]

CategoryTrackingAIReportExport=CTReportExportAI_1


In [89]:
CategoryTrackingAIReportExport=CategoryTrackingAIReportExport.drop(columns=['SyncDate&Time', 'Month', 'Year'])

In [90]:
MJNStoreMaster= StoreMaster[['LIS Store Code','Supervisor Name','Store Status']]

In [91]:
MJNStoreMaster.head()

LIS Store Code      Supervisor Name Store Status
0  MJN-Store-001  Ratih Mukasyahayati       Active
1  MJN-Store-002  Ratih Mukasyahayati       Active
2  MJN-Store-003  Ratih Mukasyahayati       Active
3  MJN-Store-005  Ratih Mukasyahayati       Active
4  MJN-Store-006  Ratih Mukasyahayati       Active

In [92]:
CategoryTrackingAIReportExport2 = pd.merge(CategoryTrackingAIReportExport,MJNStoreMaster,how='left',left_on='LIS Store Code',
                                           right_on='LIS Store Code')

In [97]:
# CategoryTrackingAIReportExport2.head(2)

In [99]:
CategoryTrackingAIReportExport2['Date'] = pd.to_datetime(CategoryTrackingAIReportExport2['Date'], dayfirst=True, format='mixed').dt.date

In [100]:
CategoryTrackingAIReportExport2.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\CategoryTrackingAIReportExporttest.csv', index=False)

In [101]:
CategoryTrackingAIReportExport2.head(2)

Visit ID        Date SupervisorCode    UserName                  FullName  \
0     15788  2025-10-06   IDN-MJNTL-06  IDN-MJN-96  Shasha Meiby Yoza Bunata   
1     15788  2025-10-06   IDN-MJNTL-06  IDN-MJN-96  Shasha Meiby Yoza Bunata   

  Client Store Code LIS Store Code Store Name Store Address Chain Code  ...  \
0           PS00016  MJN-Store-275   MM Andry   Banjarmasin        DMS  ...   
1           PS00016  MJN-Store-275   MM Andry   Banjarmasin        DMS  ...   

  AIDetectionStatus     AIdetectionreason Facings Total Facings  \
0               Yes                   NaN    13.0           105   
1                No  No products detected     NaN           105   

  Self Product Facings Detected Competition Products Detected  \
0                           3.0                           0.0   
1                           0.0                           0.0   

  Total Facings Detected Audit Status Supervisor Name Store Status  
0                    3.0      Pending   Wulan Mulyani       Active  
1                    0.0      Pending   Wulan Mulyani       Active  

[2 rows x 29 columns]